### Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

2025-02-01 11:11:12.724338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-01 11:11:12.739244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-01 11:11:12.743824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 11:11:12.755850: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


### Loading database

In [2]:
set_random_seed()

from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-01 11:11:16 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-01 11:11:16 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-01 11:11:16 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-01 11:11:16 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-01 11:11:16 [INFO]: Loaded successfully!


KeyError: 'Gender'

In [3]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing_ori = {
    "X_ori": physionet2012_dataset['test_X_ori'],
    "female_gender_test_X_ori": physionet2012_dataset['female_gender_test_X_ori'],
    "male_gender_test_X_ori": physionet2012_dataset['male_gender_test_X_ori'],
    "undefined_gender_test_X_ori": physionet2012_dataset['undefined_gender_test_X_ori'],
    "more_than_or_equal_to_65_test_X_ori":  physionet2012_dataset['more_than_or_equal_to_65_test_X_ori'],
    "less_than_65_test_X_ori": physionet2012_dataset['less_than_65_test_X_ori'],
    "ICUType_1_test_X_ori": physionet2012_dataset['ICUType_1_test_X_ori'],
    "ICUType_2_test_X_ori": physionet2012_dataset['ICUType_2_test_X_ori'],
    "ICUType_3_test_X_ori": physionet2012_dataset['ICUType_3_test_X_ori'],
    "ICUType_4_test_X_ori": physionet2012_dataset['ICUType_4_test_X_ori'],
    "classificacao_undefined_test_X_ori": physionet2012_dataset['classificacao_undefined_test_X_ori'],
    "classificacao_baixo_peso_test_X_ori": physionet2012_dataset['classificacao_baixo_peso_test_X_ori'],
    "classificacao_normal_peso_test_X_ori": physionet2012_dataset['classificacao_normal_peso_test_X_ori'],
    "classificacao_sobrepeso_test_X_ori": physionet2012_dataset['classificacao_sobrepeso_test_X_ori'],
    "classificacao_obesidade_1_test_X_ori": physionet2012_dataset['classificacao_obesidade_1_test_X_ori'],
    "classificacao_obesidade_2_test_X_ori": physionet2012_dataset['classificacao_obesidade_2_test_X_ori'],
    "classificacao_obesidade_3_test_X_ori": physionet2012_dataset['classificacao_obesidade_3_test_X_ori']
}

# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "female_gender_test_X": physionet2012_dataset['female_gender_test_X'],
    "male_gender_test_X": physionet2012_dataset['male_gender_test_X'],
    "undefined_gender_test_X": physionet2012_dataset['undefined_gender_test_X'],
    "more_than_or_equal_to_65_test_X":  physionet2012_dataset['more_than_or_equal_to_65_test_X'],
    "less_than_65_test_X": physionet2012_dataset['less_than_65_test_X'],
    "ICUType_1_test_X": physionet2012_dataset['ICUType_1_test_X'],
    "ICUType_2_test_X": physionet2012_dataset['ICUType_2_test_X'],
    "ICUType_3_test_X": physionet2012_dataset['ICUType_3_test_X'],
    "ICUType_4_test_X": physionet2012_dataset['ICUType_4_test_X'],
    "classificacao_undefined_test_X": physionet2012_dataset['classificacao_undefined_test_X'],
    "classificacao_baixo_peso_test_X": physionet2012_dataset['classificacao_baixo_peso_test_X'],
    "classificacao_normal_peso_test_X": physionet2012_dataset['classificacao_normal_peso_test_X'],
    "classificacao_sobrepeso_test_X": physionet2012_dataset['classificacao_sobrepeso_test_X'],
    "classificacao_obesidade_1_test_X": physionet2012_dataset['classificacao_obesidade_1_test_X'],
    "classificacao_obesidade_2_test_X": physionet2012_dataset['classificacao_obesidade_2_test_X'],
    "classificacao_obesidade_3_test_X": physionet2012_dataset['classificacao_obesidade_3_test_X']
}

## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori.values(), dataset_for_testing.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))# metric functions do not accpet input with NaNs, hence fill NaNs with 0


In [4]:
test_X_indicating_mask_variable = []
test_X_ori_variable = []
for i in range(len(test_X_indicating_mask)):
    test_X_indicating_mask_variable.append(test_X_indicating_mask[i].reshape(42, len(test_X_indicating_mask[i]) * 48))
    test_X_ori_variable.append(test_X_ori[i].reshape(42, len(test_X_ori[i]) * 48))

### Inicialize the models

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-01 09:33:59 [INFO]: No given device, using default device: cuda
2025-02-01 09:33:59 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250201_T093359
2025-02-01 09:33:59 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250201_T093359/tensorboard
2025-02-01 09:33:59 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 728,912


In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-01 09:34:02 [INFO]: No given device, using default device: cuda
2025-02-01 09:34:02 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250201_T093402
2025-02-01 09:34:02 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250201_T093402/tensorboard
2025-02-01 09:34:02 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 255,344


### Train the models

In [8]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-01 09:34:08 [INFO]: Epoch 001 - training loss: 0.6432, validation loss: 0.2715
2025-02-01 09:34:11 [INFO]: Epoch 002 - training loss: 0.4508, validation loss: 0.2529
2025-02-01 09:34:13 [INFO]: Epoch 003 - training loss: 0.4169, validation loss: 0.2287
2025-02-01 09:34:16 [INFO]: Epoch 004 - training loss: 0.3925, validation loss: 0.2156
2025-02-01 09:34:19 [INFO]: Epoch 005 - training loss: 0.3743, validation loss: 0.2076
2025-02-01 09:34:22 [INFO]: Epoch 006 - training loss: 0.3599, validation loss: 0.1973
2025-02-01 09:34:25 [INFO]: Epoch 007 - training loss: 0.3488, validation loss: 0.1952
2025-02-01 09:34:27 [INFO]: Epoch 008 - training loss: 0.3406, validation loss: 0.1884
2025-02-01 09:34:30 [INFO]: Epoch 009 - training loss: 0.3327, validation loss: 0.1857
2025-02-01 09:34:33 [INFO]: Epoch 010 - training loss: 0.3261, validation loss: 0.1829
2025-02-01 09:34:33 [INFO]: Finished training. The best model is from epoch#10.
2025-02-01 09:34:33 [INFO]: Saved the model to tut

In [9]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-01 09:36:59 [INFO]: Epoch 001 - training loss: 0.8452, validation loss: 0.2879
2025-02-01 09:37:49 [INFO]: Epoch 002 - training loss: 0.5927, validation loss: 0.2551
2025-02-01 09:38:37 [INFO]: Epoch 003 - training loss: 0.5456, validation loss: 0.2422
2025-02-01 09:39:25 [INFO]: Epoch 004 - training loss: 0.5236, validation loss: 0.2378
2025-02-01 09:40:14 [INFO]: Epoch 005 - training loss: 0.5097, validation loss: 0.2318
2025-02-01 09:41:03 [INFO]: Epoch 006 - training loss: 0.4991, validation loss: 0.2319
2025-02-01 09:41:50 [INFO]: Epoch 007 - training loss: 0.4900, validation loss: 0.2305
2025-02-01 09:42:37 [INFO]: Epoch 008 - training loss: 0.4827, validation loss: 0.2317
2025-02-01 09:43:25 [INFO]: Epoch 009 - training loss: 0.4765, validation loss: 0.2358
2025-02-01 09:44:12 [INFO]: Epoch 010 - training loss: 0.4718, validation loss: 0.2394
2025-02-01 09:44:12 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-02-01 09:44:12 [INFO]: Fini

### The testing stage

In [17]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
saits_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])  

teste = 'Teste'


In [22]:
saits_imputation_variable = []
for i in range(len(saits_imputation)):
    saits_imputation_variable.append(saits_imputation[i].reshape(42, len(saits_imputation[i]) * 48))

teste = 'Teste'


In [20]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [44]:
brits_imputation_variable = []
for i in range(len(brits_imputation)):
    brits_imputation_variable.append(brits_imputation[i].reshape(42, len(brits_imputation[i]) * 48))

### Calculate mean absolute error

In [40]:
testing_mae_saits_append_subgroups = []
testing_mae_saits_append_variables = []
for i in range(len(saits_imputation_variable)):
    for j in range(len(saits_imputation_variable[i])):
        testing_mae_saits_append_variables.append(calc_mae(saits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups.append(testing_mae_saits_append_variables)
    testing_mae_saits_append_variables = []
Teste = 'Teste'  

In [30]:
len(saits_imputation_variable[0][0])

115152

In [36]:
saits_imputation_variable[0][0]

array([-1.6963346 , -0.25993752, -0.28740552, ..., -0.50364804,
        0.06673649,  0.50916517], dtype=float32)

In [48]:
testing_mae_brits_append_subgroups = []
testing_mae_brits_append_variables = []
for i in range(len(brits_imputation_variable)):
    for j in range(len(brits_imputation_variable[i])):
      testing_mae_brits_append_variables.append(calc_mae(brits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups.append(testing_mae_brits_append_variables)
    testing_mae_brits_append_variables = []
Teste = 'Teste'      

In [50]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [51]:
print("SAITS - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_saits_append_subgroups[i])

SAITS - MAE
-------------
General : [0.19572739440498355, 0.20853521315798965, 0.19487407430090256, 0.2073061049144169, 0.1944357658739892, 0.19746742090645422, 0.20486614316802926, 0.22529100555207046, 0.22139870441860618, 0.21851921382191167, 0.2119795299471394, 0.20970293554553535, 0.2237109275175156, 0.19812710101598188, 0.1969537153823656, 0.1950228074439476, 0.20598885165151418, 0.21394210304939348, 0.20018607734232366, 0.20055161716443498, 0.18794757082124236, 0.21822586951340053, 0.19192918982805554, 0.19728892568022047, 0.19225807179554388, 0.2029464349340934, 0.21613127433775006, 0.19780547715775146, 0.20746290177883583, 0.19489771802802275, 0.20875520967241484, 0.20989458246066348, 0.21454475577605597, 0.202597888736192, 0.1962651487012549, 0.1985894046155526, 0.19656423254722666, 0.21395801079093715, 0.20067152306791095, 0.21295627724084598, 0.2009485002710331, 0.21180076670475223]
Female : [0.18286259372835592, 0.19643363068126918, 0.1982360345010649, 0.23486070362116754, 

In [52]:
print("BRITS - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_brits_append_subgroups[i])

BRITS - MAE
-------------
General : [0.1962358019693381, 0.20542357592674076, 0.1979216866218381, 0.21234099306466156, 0.19585661894886525, 0.2080356559322864, 0.21492250081022543, 0.2187574466914169, 0.22451208701626463, 0.20534600951292856, 0.20161345757400367, 0.21571329323125008, 0.22548847222615792, 0.19562882462622477, 0.1906580070514125, 0.192236055292469, 0.2027099126832753, 0.20289585947022934, 0.2021046325940434, 0.20268077647893198, 0.19416009340782187, 0.21771963348665715, 0.19259169838636778, 0.19840813990637, 0.19654472241928775, 0.20458692540807955, 0.208304236080579, 0.198183506126367, 0.20439084666758744, 0.19867903061675873, 0.21813141793117619, 0.20837618191478277, 0.20929837620746616, 0.19711017965951236, 0.19789701798789477, 0.2074465156434963, 0.20109083397960958, 0.21883669923438365, 0.20014034149217563, 0.21575077429591716, 0.20188151679612304, 0.21318694504483018]
Female : [0.18623341232781088, 0.19501365077846933, 0.19024424406610316, 0.24286887087861292, 0.20